In [1]:
# Get all the players from Sleeper and store the pertinent information in a SQLite database
import requests
import sqlite3
import json
from sleeper import get_all_players, get_users_in_league, get_all_rosters
from manage import clean_player_data
from config import LEAGUE_LIST
import pandas as pd
LEAGUE_ID = "1234567890123456789"
SLEEPER_API = "https://api.sleeper.app/v1/players/nfl"

with open('../data/players.json', 'r') as file:
        player_json = json.load(file)

In [16]:
# Get all the players from Sleeper
players_json = get_all_players()


In [19]:
# Get all the players from Sleeper
players_json = get_all_players()
players_df = clean_player_data(players_json)

# Create a connection to the database
conn = sqlite3.connect('../data/league_db.db')
# Insert the data into the database
players_df.to_sql('players', conn, if_exists='replace', index=True)
conn.commit()
conn.close()

# Insert the coaches into the database

In [2]:
# Create a connection to the database
conn = sqlite3.connect('../data/league_db.db')
# Remove the coaches table if it exists already
conn.execute("DROP TABLE IF EXISTS coaches")
# Get all the coaches in each of the leagues
for league in LEAGUE_LIST:
    coaches_df = get_users_in_league(LEAGUE_LIST[league])
    coaches_df.to_sql('coaches', conn, if_exists='append', index=False)

# Update the team names if they aren't already set by combining the display name and Team like "Team" + Display Name from the row
conn.execute("UPDATE coaches SET team_name = 'Team ' + display_name WHERE team_name IS NULL")

# Close the connection
conn.close()

# Now get the drafts from every league and put these in the database

In [3]:
# Get the drafts for each league
# Create a connection to the database
conn = sqlite3.connect('../data/league_db.db')
# Insert the data into the database
conn.execute("DROP TABLE IF EXISTS roster")
for league in LEAGUE_LIST:
    rosters = get_all_rosters(LEAGUE_LIST[league])
    # Expand the players columns into individual rows keeping just the player_id, the owner_id, and league_id
    rosters_df = rosters.explode('players')[['players','owner_id','league_id']].rename(columns={'players':'player_id'})
    
    rosters_df.to_sql('rosters', conn, if_exists='append', index=False)
# Close the connection
conn.close()


In [40]:
conn = sqlite3.connect('../data/league_db.db')
rows = conn.execute(f'Select p.full_name, p.college, p.years_exp, p.team, p.position, c.display_name, c.league_id from players p join rosters r on p.player_id = r.player_id '
             'join coaches c on r.owner_id = c.user_id and r.league_id = c.league_id ').fetchall()
# turn the rows into a dataframe
df = pd.DataFrame(rows, columns=['full_name','college', 'years_experience','team', 'position', 'coach','league_id'])
# Output the data to a csv file
df.to_csv('../data/players_rostered.csv', index=False)
df

,full_name,college,years_experience,team,position,display_name,league_id
0,Jonathan Mingo,Ole Miss,1.0,CAR,WR,ChiJV,1060410179696533504
1,Rashee Rice,SMU,1.0,KC,WR,ChiJV,1060410179696533504
2,Roschon Johnson,Texas,1.0,CHI,RB,ChiJV,1060410179696533504
3,Brenden Rice,USC,0.0,LAC,WR,ChiJV,1060410179696533504
4,Luke McCaffrey,Rice,0.0,WAS,WR,ChiJV,1060410179696533504
...,...,...,...,...,...,...,...
1174,Kenneth Walker,Michigan State,2.0,SEA,RB,WestCoastNonsense,1104635110046986240
1175,Brock Purdy,Iowa State,2.0,SF,QB,WestCoastNonsense,1104635110046986240
1176,Anthony Richardson,Florida,1.0,IND,QB,WestCoastNonsense,1104635110046986240
1177,C.J. Stroud,Ohio State,1.0,HOU,QB,WestCoastNonsense,1104635110046986240


In [41]:
# Output the data to a csv file
df.to_csv('../data/players_rostered.csv', index=False)